In [1]:
import os, shutil, glob
import pysam
from Bio import SeqIO
import os
import time
import vcf
import math
from cigar import Cigar
import hashlib
protein = {"TTT" : "F", "CTT" : "L", "ATT" : "I", "GTT" : "V",
           "TTC" : "F", "CTC" : "L", "ATC" : "I", "GTC" : "V",
           "TTA" : "L", "CTA" : "L", "ATA" : "I", "GTA" : "V",
           "TTG" : "L", "CTG" : "L", "ATG" : "M", "GTG" : "V",
           "TCT" : "S", "CCT" : "P", "ACT" : "T", "GCT" : "A",
           "TCC" : "S", "CCC" : "P", "ACC" : "T", "GCC" : "A",
           "TCA" : "S", "CCA" : "P", "ACA" : "T", "GCA" : "A",
           "TCG" : "S", "CCG" : "P", "ACG" : "T", "GCG" : "A",
           "TAT" : "Y", "CAT" : "H", "AAT" : "N", "GAT" : "D",
           "TAC" : "Y", "CAC" : "H", "AAC" : "N", "GAC" : "D",
           "TAA" : "STOP", "CAA" : "Q", "AAA" : "K", "GAA" : "E",
           "TAG" : "STOP", "CAG" : "Q", "AAG" : "K", "GAG" : "E",
           "TGT" : "C", "CGT" : "R", "AGT" : "S", "GGT" : "G",
           "TGC" : "C", "CGC" : "R", "AGC" : "S", "GGC" : "G",
           "TGA" : "STOP", "CGA" : "R", "AGA" : "R", "GGA" : "G",
           "TGG" : "W", "CGG" : "R", "AGG" : "R", "GGG" : "G" 
           }  
def translateToProtMap(dna):
    map_protein={}
    count=0
    if len(dna)%3 == 0:
        for i in range(0, len(dna), 3):
            codon = dna[i:i + 3]
            
            count=count+1
            map_protein[i+1]={'codon':str(codon),'p':protein[dna[i:i+3]],'pos':count}  
    return map_protein
def translateToProt(dna):    
    protein_sequence = ""
    if len(dna)%3 == 0:
        for i in range(0, len(dna), 3):
            codon = dna[i:i + 3]
            protein_sequence+= protein[codon]
    
    return protein_sequence
def GlobalAlignment(v,w,score,sigma):
    if v=='' or w=='':
        return '',0
    s=[[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    bk=[[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    s[0][0]=0
    for i in range(1,len(v)+1):
        s[i][0]=s[i-1][0]-sigma
    for j in range(1,len(w)+1):
        s[0][j]=s[0][j-1]-sigma
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # s[i][j]=max(s[i-1][j]-sigma,s[i][j-1]-sigma,s[i-1][j-1]+score[v[i-1]][w[j-1]])
            if not v[i-1]==w[j-1]:
                s[i][j]=max(s[i-1][j]-sigma,s[i][j-1]-sigma,s[i-1][j-1]-1)
            else:
                s[i][j]=s[i-1][j-1]+score
            if s[i][j]==s[i-1][j]-sigma:
                bk[i][j]=1
            elif s[i][j]==s[i][j-1]-sigma:
                bk[i][j]=2
            else:
                bk[i][j]=3
    AlignV=''
    AlignW=''
    i=len(v)
    j=len(w)
    while True:
        if i>0 and j>0:
            if bk[i][j] ==3:

                AlignV=v[i-1]+AlignV
                AlignW=w[j-1]+AlignW
                i=i-1
                j=j-1
            elif bk[i][j] ==1:

                AlignW='-'+AlignW
                AlignV=v[i-1]+AlignV
                i =i-1
            else:

                AlignW=w[j-1]+AlignW
                AlignV='-'+AlignV
                j =j-1
        elif i>0:
            AlignW='-'+AlignW
            AlignV=v[i-1]+AlignV
            i =i-1
        elif j>0:
            AlignW=w[j-1]+AlignW
            AlignV='-'+AlignV
            j =j-1
        else:
            break
    mm=[]
  
    for i in range(len(AlignV)):
        if not AlignV[i]==AlignW[i]:
           
            mm.append({'pos':i+1,'ref':AlignV[i],'alt':AlignW[i]})       
    return mm,AlignV,AlignW
def calVariants(ref_fa,query_fa,output):
    '''Index reference genome, call aligment with BWA and convert to vcf'''
    temp_folder=output+"/temp"
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)
 
    #alignment
    cmd = 'bwa mem {ref} {query} > {output}/aln-se.sam'.format(
        ref=ref_fa,
        query=query_fa,
        output=temp_folder
    )
    os.system(cmd)
    cmd='minimap2 -ax asm5 {ref} {query} > {output}/aln.sam'.format(ref=ref_fa, query=query_fa,
        output=temp_folder)
    #os.system(cmd)
    #filter duplicated aligment
    cmd = 'samtools view -hF 2308 {output}/aln-se.sam > {output}/aln-filtered.sam'.format(
        output=temp_folder
    )
    #os.system(cmd)
    #convert to bam
    #cmd = 'samtools view -S -b {output}/aln-filtered.sam > {output}/aln-filtered.bam'.format(
    cmd = 'samtools view -S -b {output}/aln-se.sam > {output}/aln-se.bam'.format(
        output=temp_folder
    )
    os.system(cmd)
    #sort BAM for SNP calling
    #cmd = 'samtools sort {output}/aln-filtered.bam > {output}/aln-filtered-sorted.bam'.format(
    cmd = 'samtools sort {output}/aln-se.bam > {output}/aln-sorted.bam'.format(
        output=temp_folder
    )
    os.system(cmd)
    
    #generate raw bcf
    #cmd = 'samtools mpileup -g -f {ref} {output}/aln-filtered-sorted.bam | bcftools call -mv -Ov > {output}/variants.vcf'.format(
    cmd = 'samtools mpileup -g  -f {ref} {output}/aln-sorted.bam | bcftools call -mv -Ov > {output}/variants.vcf'.format(
    
        ref=ref_fa,
        output=temp_folder
    )
    os.system(cmd)
    
    #generate vcf
    '''
    
    cmd = 'bcftools view -bvcg {output}/raw.bcf > {output}/var.bcf'.format(

        output=temp_folder
    )
    os.system(cmd)
    #generate vcf
    cmd = 'bcftools view {output}/var.bcf | vcfutils.pl varFilter - > {output}/var-final.vcf'.format(

            output=temp_folder
        )
    os.system(cmd)'''
    cmd='freebayes -C 1 -f {ref} {output}/aln-se-sorted.bam> {output}/var.vcf'.format( ref=ref_fa,
        output=temp_folder)
    #os.system(cmd)
   
   
    return  temp_folder+'/variants.vcf', temp_folder+'/aln-se.sam'
def translateDNAMuToProMu(map_prot, pos, ref, aln):
    str_prot_mul=''
    p_start=pos
    order=0
    note=''
    if pos%3==0:
        p_start=p_start-2
        order=2
    elif pos%3==2:
        p_start=p_start-1
        order=1
    if len(ref)==1 and len(aln)==1:
        ref_p=map_prot[p_start]['p']
        codon=map_prot[p_start]['codon']
        list1 = list(codon)
        list1[order]=aln
        codon=''.join(list1)
        aln_p=protein[codon]
        
        if not ref_p==aln_p:
            str_prot_mul=ref_p+str(map_prot[p_start]['pos'])+aln_p+','
    else :
       
        num_diff=abs(len(ref)-len(aln))
        if not  num_diff%3==0:
            str_prot_mul=str(map_prot[p_start]['pos'])+'frameshift'
        else:
            #in-frame
            #number of codon effected
            diff=pos-p_start
            num_b=diff+len(ref)
            num_c=1
            if num_b%3==0:
                num_c=num_b/3
            else:
                num_c=math.floor(num_b/3)+1
            print('numc:'+str(num_c))
            str_codons=''
            for c in range(int(num_c)):
                str_codons+=map_prot[p_start+c*3]['codon']
            #split and build new str codon
            new_str_codons=str_codons[:diff]+aln  +    str_codons[diff+len(ref):]     
            old_p=translateToProt(str_codons)
            new_p=translateToProt(new_str_codons)  
           
            mm,aop,anp=GlobalAlignment(old_p,new_p,1,1)   
            print(ref)
            print(aln)
            print(str_codons)
            print(aop)
            print(anp)
            print(mm)
            p=0
            map_pos_to_pos_align={}
            for i in range(len(aop)):
                if not aop[i]=='-':                
                    map_pos_to_pos_align[i]=map_prot[p_start+p*3]
                    p=p+1
            print(map_pos_to_pos_align)
            str_prot_mul=''
            if len(mm)>0:
                for m in mm:
                    if m['alt']=='-':
                        str_prot_mul=str_prot_mul+m['ref']+str(map_pos_to_pos_align[m['pos']-1]['pos'])+m['alt']+',' 
                    elif m['ref']=='-':
                        cursor=m['pos']-1
                        lastp=m['pos']-1
                        while cursor>=0:
                            if not aop[cursor]== '-':
                                lastp=cursor
                                break
                            cursor=cursor-1
                        left_p=None
                        if lastp in map_pos_to_pos_align.keys():
                            left_p=map_pos_to_pos_align[lastp]
                        else:
                            left_p=map_prot[p_start-3]
                        cursor=m['pos']-1
                        firstp=m['pos']-1
                        
                        while cursor<len(aop):
                            if not aop[cursor]== '-':
                                firstp=cursor
                                break
                            cursor=cursor+1
                        right_p=None
                        if firstp in map_pos_to_pos_align.keys():
                            right_p=map_pos_to_pos_align[firstp]
                        else:
                            print(left_p)
                            right_p=map_prot[left_p['pos']*3-2+3]
                        str_prot_mul=str_prot_mul+left_p['p']+str(left_p['pos'])+'_'+right_p['p']+str(right_p['pos'])+'ins'+m['alt']+','
                    else:
                        str_prot_mul=str_prot_mul+m['ref']+str(map_pos_to_pos_align[m['pos']-1]['pos'])+m['alt']+','
                
            
  
        
    return str_prot_mul[:-1]
def readVCF(vcffile):
    arrmul=[]
    vcf_reader = vcf.Reader(open(vcffile, 'r'))
    for record in vcf_reader:
        arrmul.append({'pos':record.POS,'ref':record.REF,'alts':record.ALT})
    return arrmul
def readSAM(samfile):
    hit=[]
    with open(samfile) as f:
        lines=f.readlines()
        for i in range(len(lines)):
            if not lines[i].startswith('@'):
                token=lines[i].split('\t')
                if token[1]=='4':
                    continue              
                hit.append({'chr':token[0],'ss':token[3],'cigar':token[5],'seq':token[9]})
               
   
    return hit      
def getGeneLocation(hit,gene):
    #parse CIGAR string
    loc={}
    loc['note']=''
    list_hit=[]
    for h in hit:
        hit={}
        cigar=Cigar(h['cigar'])
        items=list(cigar.items())
        if items[0][1]=='S'and items[-1][1]=='S':
            hit['seq']=h['seq'][items[0][0]:-items[-1][0]]
        elif items[0][1]=='H'and items[-1][1]=='H':
            hit['seq']=h['seq']
        else:
            hit['seq']=h['seq']
        send=int(h['ss'])
        for item in items:
            if item[1]=='M':
                send=send+int(item[0])
            if item[1]=='D':
                send=send+int(item[0])
            if item[1]=='I':
                send=send-int(item[0])
        hit['pos']= int(items[0][0]  )
        hit['ss']=int(h['ss'])
        hit['send']=send
        list_hit.append(hit)
    list_hit_sorted=sorted(list_hit, key=lambda k: k['ss']) 
    if not len(list_hit_sorted)>0:
        loc['consensus']=''
        loc['hit']=[]
        loc['note']='Not found'
        return loc
    scafold=[list_hit_sorted[0]]
    for i in range(len(list_hit_sorted)):
        if list_hit_sorted[i]['ss']>scafold[-1]['ss']+len(scafold[-1]['seq']):
            scafold.append(list_hit_sorted[i])
    if len(scafold)<len(list_hit_sorted):
        loc['note']="Multiple sequences found"
    loc['hit']=list_hit_sorted
    loc['consensus']=makeConsensus(scafold,gene)
    #make consensus sequence from scafold
        
    print(loc)
    return loc
def makeConsensus(scafold,gene):
    list_remain=[]
    first_seq=''
    if scafold[0]['ss']>1:
        first_seq=gene[:scafold[0]['ss']]
    
    for i in range (len(scafold)-1):
        list_remain.append(gene[scafold[i]['send']-1:scafold[i+1]['ss']-1])
    last_seq=gene[scafold[-1]['send']:]
    consensus=''+first_seq
    for i in range (len(scafold)-1):
        consensus=consensus+scafold[i]['seq']+list_remain[i]
    consensus=consensus+last_seq
    return consensus
def makePhylo(sequence_type,output,gene_ref):
    if not os.path.exists(output+'/phylo'):
        os.makedirs(output+'/phylo')
    print(sequence_type)
    for k in sequence_type.keys():
        f = open(output+'/phylo/'+sequence_type[k]['type']+'.fasta', "w")
        f.write(">"+sequence_type[k]['type']+'\n')
        f.write(str(sequence_type[k]['seq']))
        f.close()
    cmd = 'parsnp -r {} -d {} -o {} -p {}'.format(
        gene_ref,
        output+'/phylo',
        output, 5)
    os.system(cmd)
def pipeline(gene_ref,sample_seqs,output):
    str_dna=''
    with open('data2/gene.fna') as handle:
        for record in SeqIO.parse(handle, "fasta"):
            str_dna=record.seq
    m_prot=translateToProtMap(str_dna)
       #indexing reference
    isIndex=False 
    if not isIndex:
        cmd='bwa index '+gene_ref
        os.system(cmd)
        cmd='samtools faidx '+gene_ref
        os.system(cmd)
    gene_profiles={}
    sequence_type={}
    report=[]
    with open(sample_seqs) as handle:
        for record in SeqIO.parse(handle, "fasta"):
            record_f=record.id.replace('/','_')
            gene_profiles[record_f]={}
            with open(record_f+".fasta", "w") as output_handle:
                SeqIO.write(record, output_handle, "fasta")
            if not os.path.exists(os.path.join(output,record_f)):
                os.makedirs(os.path.join(output,record_f))          
            vcf,sam=calVariants(gene_ref,record_f+".fasta",os.path.join(output,record_f))
            mutations=readVCF(vcf)
            str_mut=''
            str_dna_mut=''
            for mut in mutations:
                for alt in mut['alts']:
                    str_mut=str_mut+translateDNAMuToProMu(m_prot,mut['pos'],mut['ref'],str(alt))+','
                    
            gene_profiles[record_f]['aaSubstitutions']=[str_mut]     
            sam=readSAM(sam)
            loc=getGeneLocation(sam,str_dna)
            print(record_f+":"+str_mut)
            str_md5 = hashlib.md5(str(loc['consensus']).encode()).hexdigest()
            stype=1
            if len(loc['consensus'])>0:
                if not str_md5 in sequence_type.keys():
                    sequence_type[str_md5]={}
                    sequence_type[str_md5]['type']='Type'+str(stype)
                    sequence_type[str_md5]['seq']=loc['consensus']
                    stype=stype+1
                    
                with open(os.path.join(output,record_f)+"/consensus.fasta", "w") as output_handle:
                
                    record.seq=loc['consensus']
                    SeqIO.write(record, output_handle, "fasta")
            
                report.append({'sid':record.id,'aaSubstitutions':str_mut,'hit':len(loc['hit']),'seq':sequence_type[str_md5]['type'],'note':loc['note']})
    makePhylo(sequence_type,output,gene_ref)
    print(report)
    print(len(sequence_type))
start = time.time()    
pipeline('data2/gene.fna','data2/collection.fasta','data2')
'''str_dna='TTGTCGACGCACGAT'
new_dna='TTGACGCACGAT'
print(translateToProt(str_dna))
print(translateToProt(new_dna))
print(translateToProt('TTGTCGACG'))
print(translateToProt('TTGTCG'))
print(translateToProt('TTG'))
m_prot=translateToProtMap(str_dna)
print(m_prot)
print(translateDNAMuToProMu(m_prot,4,'T','TTTT'))  '''                              
end = time.time()
print(end - start)

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}, {'seq': 'AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGTTTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTGGTGGTTTTAATTTTTCACAAATATTACCAGATCCATCAAAACCAAGCAAGAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATGCTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGAGACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTG', 'pos': 23812, 'ss': 2251, 'send': 2584}, {'seq': 'CAGTCAGCACCTCATGGTGTAGTCTTCTTGCATGTGACTTATGTCCCTGCACAAGAAAAGAACTTCACAACTGCTCCTGCCATTTGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAATGGCACACACTGGTTTGTAACACAAAGGAATTTTTATGAAC

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}, {'seq': 'AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGTTTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTGGTGGTTTTAATTTTTCACAAATATTACCAGATCCATCAAAACCAAGCAAGAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATGCTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGAGACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTG', 'pos': 23812, 'ss': 2251, 'send': 2584}, {'seq': 'GTTGTTCTTGTGGATCCTGCTGCAAATTTGATGAAGACGACTCTGAGCCAGTGCTCAAAGGAGTCAAATTACATTACACATAA', 'pos': 25301, 'ss': 3740, 'send': 3823}], 'consensus': Seq('ATGTTTGTTTTTCTTGTTTTATTGCCACT

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTTAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCATGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}, {'seq': 'AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGTTTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTGGTGGTTTTAATTTTTCACAAATATTACCAGATCCATCAAAACCAAGCAAGAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATGCTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGAGACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTG', 'pos': 23812, 'ss': 2251, 'send': 2584}, {'seq': 'GTTGTTCTTGTGGATCCTGCTGCAAATTTGATGAAGACGACTCTGAGCCAGTGCTCAAAGGAGTCAAATTACATTACACATAA', 'pos': 25301, 'ss': 3740, 'send': 3823}], 'consensus': Seq('ATGTTTGTTTTTCTTGTTTTATTGCCACT

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}, {'seq': 'AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGTTTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTGGTGGTTTTANNTTTTCACAAATATTACCAGATCCATCAAAACCAAGCAAGAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATGCTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGAGACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTG', 'pos': 23812, 'ss': 2251, 'send': 2584}, {'seq': 'CAGTCAGCACCTCATGGTGTAGTCTTCTTGCATGTGACTTATGTCCCTGCACAAGAAAAGAACTTCACAACTGCTCCTGCCATTTGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAATGGCACACACTGGTTTGTAACACAAAGGAATTTTTATGAAC

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}, {'seq': 'AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGTTTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTGGTGGTTTTAATTTTTCACAAATATTACCAGATCCATCAAAACCAAGCAAGAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATGCTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGAGACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTG', 'pos': 23812, 'ss': 2251, 'send': 2584}], 'consensus': Seq('ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTT...TAA')}
barcode31_ARTIC_nanopolish:D614G,
{'note': '', 'hit': [{'seq': 'ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTA

...

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}, {'seq': 'AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGTTTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTGGTGGTTTTANNTTTTCACAAATATTACCAGATCCATCAAAACCAAGCAAGAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATGCTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGAGACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTG', 'pos': 23812, 'ss': 2251, 'send': 2584}, {'seq': 'CAGTCAGCACCTCATGGTGTAGTCTTCTTGCATGTGACTTATGTCCCTGCACAAGAAAAGAACTTCACAACTGCTCCTGCCATTTGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAATGGCACACACTGGTTTGTAACACAAAGGAATTTTTATGAAC

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}, {'seq': 'AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGTTTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTGGTGGTTTTANNTTTTCACAAATATTACCAGATCCATCAAAACCAAGCAAGAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATGCTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGAGACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTG', 'pos': 23812, 'ss': 2251, 'send': 2584}, {'seq': 'CAGTCAGCACCTCATGGTGTAGTCTTCTTGCATGTGACTTATGTCCCTGCACAAGAAAAGAACTTCACAACTGCTCCTGCCATTTGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAATGGCACACACTGGTTTGTAACACAAAGGAATTTTTATGAAC

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}], 'consensus': Seq('ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTT...TAA')}
barcode87_ARTIC_nanopolish:D614G,
{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}], 'consensus': Seq('ATGT

{'note': '', 'hit': [{'seq': 'AAAAGTCTACTAATTTGGTTAAAAACAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTACTGAGTCTAACAAAAAGTTTCTGCCTTTCCAACAATTTGGCAGAGACATTGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTCTAATGTTTTTCAAACACGT', 'pos': 23144, 'ss': 1583, 'send': 1939}, {'seq': 'AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGTTTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTGGTGGTTTTANNTTTTCACAAATATTACCAGATCCATCAAAACCAAGCAAGAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATGCTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGAGACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTG', 'pos': 23812, 'ss': 2251, 'send': 2584}, {'seq': 'CAGTCAGCACCTCATGGTGTAGTCTTCTTGCATGTGACTTATGTCCCTGCACAAGAAAAGAACTTCACAACTGCTCCTGCCATTTGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAATGGCACACACTGGTTTGTAACACAAAGGAATTTTTATGAAC

[{'sid': 'barcode01/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 3, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode02/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 4, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode03/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 2, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode04/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 1, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode05/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 4, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode06/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 3, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode07/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 3, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode08/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 4, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode09/ARTIC/nanopolish', 'aaSubstitutions': 'D614G,', 'hit': 3, 'seq': 'Type1', 'note': ''}, {'sid': 'barcode10/ARTIC/nanopolish', 'aaSubs